In [4]:
#读写文件
import torch
from torch import nn
from torch.nn import functional as F
#save tensor
X = torch.arange(4)
torch.save(X,"x_file")

X = torch.load("x_file")


In [10]:
y = torch.zeros(4)
torch.save([x, y],'x-files')
x2, y2 = torch.load('x-files')
(x2, y2)

NameError: name 'x' is not defined

In [12]:
mydict = {'x': X, 'y': y}
torch.save(mydict, 'mydict')
mydict2 = torch.load('mydict')
mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

In [13]:
#save model

In [14]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.output = nn.Linear(256, 10)

    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))

net = MLP()
X = torch.randn(size=(2, 20))
Y = net(X)

In [15]:
torch.save(net.state_dict(), 'mlp.params')#实际上存的权重参数

In [17]:
clone = MLP()#同时要复制网络
clone.load_state_dict(torch.load('mlp.params'))
clone.eval()

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

In [18]:
Y_clone = clone(X)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])

In [20]:
#模型构造
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[ 0.3224, -0.3817,  0.1460, -0.0752,  0.0722, -0.2322, -0.0559,  0.0830,
         -0.3249, -0.1640],
        [ 0.3072, -0.3354,  0.1977,  0.0066, -0.0370, -0.0660,  0.0892,  0.1725,
         -0.1927, -0.0996]], grad_fn=<AddmmBackward0>)

In [21]:
class MLP(nn.Module):#自己的网络继承自Module
    # 用模型参数声明层。这里，我们声明两个全连接的层
    def __init__(self):
        # 调用MLP的父类Module的构造函数来执行必要的初始化。
        # 这样，在类实例化时也可以指定其他函数参数，例如模型参数params（稍后将介绍）
        super().__init__()
        self.hidden = nn.Linear(20, 256)  # 隐藏层
        self.out = nn.Linear(256, 10)  # 输出层

    # 定义模型的前向传播，即如何根据输入X返回所需的模型输出
    def forward(self, X):
        # 注意，这里我们使用ReLU的函数版本，其在nn.functional模块中定义。
        return self.out(F.relu(self.hidden(X)))

In [22]:
net = MLP()
net(X)

tensor([[ 0.0078, -0.0817, -0.0841,  0.0037, -0.2199,  0.0881,  0.1784,  0.0367,
         -0.2293, -0.1512],
        [-0.1405, -0.1272,  0.1362,  0.0430, -0.0765,  0.1130,  0.2354,  0.0604,
         -0.0595, -0.0145]], grad_fn=<AddmmBackward0>)

In [23]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[-0.0074],
        [ 0.0512]], grad_fn=<AddmmBackward0>)

In [24]:
print(net[2].state_dict())

OrderedDict([('weight', tensor([[-0.0755,  0.2172,  0.3121,  0.1394, -0.1795, -0.2196,  0.0741,  0.3374]])), ('bias', tensor([-0.0554]))])


In [25]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.0554], requires_grad=True)
tensor([-0.0554])


In [29]:
!nvidia-smi

Tue Jan  3 16:55:26 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 461.56       Driver Version: 461.56       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 306... WDDM  | 00000000:01:00.0  On |                  N/A |
| N/A   38C    P0    28W /  N/A |   2871MiB /  6144MiB |     10%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [30]:
#默认cpu
import torch
from torch import nn

torch.device('cpu'), torch.device('cuda'), torch.device('cuda:1')

(device(type='cpu'), device(type='cuda'), device(type='cuda', index=1))

In [31]:
torch.cuda.device_count()

1

In [32]:
def try_gpu(i=0):  #@save
    """如果存在，则返回gpu(i)，否则返回cpu()"""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

def try_all_gpus():  #@save
    """返回所有可用的GPU，如果没有GPU，则返回[cpu(),]"""
    devices = [torch.device(f'cuda:{i}')
             for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]

try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0)])

In [33]:
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

In [34]:
X = torch.ones(2, 3, device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

In [35]:
Y = torch.rand(2, 3, device=try_gpu(1))
Y

tensor([[0.6349, 0.4100, 0.8277],
        [0.6255, 0.2107, 0.0857]])

In [37]:
Y + Z

NameError: name 'Z' is not defined

In [38]:
net = nn.Sequential(nn.Linear(3, 1))
net = net.to(device=try_gpu())

In [39]:
net(X)

tensor([[-0.0735],
        [-0.0735]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [40]:
net[0].weight.data.device

device(type='cuda', index=0)